## Best Practices 
1. preprocessing and cleaning 
2. Train Test Split 
3. BOW,TFIDF,WORD2VEC 
4. Train Ml algo 


In [76]:
import pandas as pd 
import re 
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [49]:
## Read The Data 
df=pd.read_csv("all_kindle_review .csv")
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [50]:
df=df[['reviewText','rating']]

In [51]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [52]:
df.shape

(12000, 2)

In [53]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [54]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [55]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [56]:
### Preprocessing And The Data 
## posative review is 1 and the neg is 0 
df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)

In [57]:
df['rating'].unique()

array([1, 0], dtype=int64)

In [58]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [59]:
##Lower All the Cases 
df['reviewText']=df['reviewText'].str.lower()

In [60]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [61]:
### Clean The Data Removing Special Characters 
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^A-Z a-z 0-9]+','',x))
### Remove Stop Words 
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
### Remove The Url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
##Remove the Html Tags 
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x,'lxml').get_text())
## Remove any additional Spaces 
df['reviewText']=df['reviewText'].apply(lambda x:" ".join(x.split()))

In [62]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [63]:
lemmetizer=WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmetizer.lemmatize(word) for word in text.split()])

In [64]:
df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_words(x))

In [65]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [66]:
X=df['reviewText']
y=df['rating']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.20, random_state=42,stratify=y)

In [67]:
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [70]:
tfidf=TfidfVectorizer()
X_train_tf=tfidf.fit_transform(X_train).toarray()
X_test_tf=tfidf.transform(X_test).toarray()

In [73]:
nb_bow=MultinomialNB()

nb_bow.fit(X_train_bow,y_train)


MultinomialNB()

In [75]:
y_predict_bow=nb_bow.predict(X_test_bow)

In [77]:
accuracy_score(y_predict_bow,y_test)

0.8479166666666667

In [79]:
print(confusion_matrix(y_predict_bow,y_test))

[[ 574  139]
 [ 226 1461]]


In [80]:
print(classification_report(y_predict_bow,y_test))

              precision    recall  f1-score   support

           0       0.72      0.81      0.76       713
           1       0.91      0.87      0.89      1687

    accuracy                           0.85      2400
   macro avg       0.82      0.84      0.82      2400
weighted avg       0.86      0.85      0.85      2400



In [74]:
nb_tfidf=MultinomialNB()
nb_tfidf.fit(X_train_bow,y_train)

MultinomialNB()

In [82]:
y_predict_tfidf=nb_tfidf.predict(X_test_bow)

In [84]:
accuracy_score(y_predict_bow,y_test)

0.8479166666666667

In [85]:
print(confusion_matrix(y_predict_bow,y_test))

[[ 574  139]
 [ 226 1461]]


In [86]:
print(classification_report(y_predict_bow,y_test))

              precision    recall  f1-score   support

           0       0.72      0.81      0.76       713
           1       0.91      0.87      0.89      1687

    accuracy                           0.85      2400
   macro avg       0.82      0.84      0.82      2400
weighted avg       0.86      0.85      0.85      2400

